In [2]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sktime.transformations.panel.catch22 import Catch22
import math

In [2]:
# Import
path = r'switch_data/antibiotic_vitals.csv'
antibiotic_vitals_pivoted_4 = pd.read_csv(path)
antibiotic_vitals_pivoted_4['hours_grouped'] = antibiotic_vitals_pivoted_4['hours_grouped'].astype(int) # Needs to be type int for c22 to work

In [3]:
# Remove those without enough data
c22_other = antibiotic_vitals_pivoted_4[antibiotic_vitals_pivoted_4['SPELL_IDENTIFIER'].isin((antibiotic_vitals_pivoted_4.groupby('SPELL_IDENTIFIER').size() > 8).where(lambda x : x == True).dropna().reset_index()['SPELL_IDENTIFIER'].to_list())]
# Set index
c22_other.set_index(['SPELL_IDENTIFIER', 'date', 'hours_grouped'] ,inplace=True, drop=True)

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
def c22_other_fun(df):
    c22 = Catch22(catch24=True)
    master_df = pd.DataFrame()
    overlord_df = pd.DataFrame()
    for x in range(len(df.columns)): # Iterate through columns sas this c22 does not work on multiple columns
        print('column number:', x)
        working_df = df.iloc[:, x]
        working_df = working_df.to_frame()
        for column_name in working_df:
            master_df = pd.DataFrame()
            master_df2 = pd.DataFrame()
            for stay_id, new_df in working_df[[column_name]].groupby(level=0, dropna=False): # Iterate through stays

                n_list = list(range(1, math.ceil(len(new_df)/4)+1))
                for n in range(1, math.ceil(len(new_df)/4)+1): # Do this to understand how many 'days' / sets of data we will have for each stay beyond 24 and 48 hours
                    # Ignore first 48 hours as calculated seperatly
                    if n == 1:
                        continue
                    elif n == 2:
                        continue
                    # This is for exceptional cases where we want hours grouped 1 for last day and the spell starts on hours grouped 2
                    elif (new_df.reset_index().iloc[0]['hours_grouped'] == 2) and ((n == (len(new_df)/4)) and (n == n_list[-1])):
                        exit_flag = True
                        flag_2 = False
                        flag_1 = False
                        # Get index of nth 2 hours grouped value
                        try:
                            index_value = new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index][new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index]['hours_grouped'] == 2].index.values[0]
                            new_df2 = new_df.iloc[:index_value+1]
                            exit_flag = False
                            flag_2 = True
                        #  Look at nth 1 hours grouped  
                        except:
                            pass

                        try:
                            index_value = new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index][new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index]['hours_grouped'] == 1].index.values[0]
                            new_df3 = new_df.iloc[:index_value+1]
                            exit_flag = False
                            flag_1 = True
                        except:
                            pass
                        
                        if exit_flag == True:
                            continue
                        
                        if flag_2 == True:
                            new_df2_2 = new_df2.tail(4).reset_index(drop=True).dropna()
                            new_df2_3 = new_df2.reset_index(drop=True).dropna()
                    
                            # C22 for latest day
                            if len(new_df2_2) <= 2: # c22 only works with 3 or more timepoints
                                transformed_data = pd.DataFrame()
                                np_data = new_df2_2.to_numpy()
                                if not np_data.size == 0:
                                    transformed_data['22'] = np.mean(np_data)
                                    transformed_data['23'] = np.std(np_data) 
                            else: 
                                transformed_data = c22.fit_transform(new_df2_2)

                            transformed_data = transformed_data.add_prefix(column_name)           
                            transformed_data.insert(0, 'SPELL_IDENTIFIER', stay_id)
                            transformed_data.insert(1, 'date', new_df2.reset_index().iloc[-1]['date'])

                            # C22 for all data to date for stay
                            if len(new_df2_3) <= 2: # c22 only works with 3 or more timepoints
                                transformed_data2 = pd.DataFrame()
                                np_data = new_df2_3.to_numpy()
                                if not np_data.size == 0:
                                    transformed_data2['22'] = np.mean(np_data)
                                    transformed_data2['23'] = np.std(np_data)
                            else: 
                                transformed_data2 = c22.fit_transform(new_df2_3)

                            transformed_data2 = transformed_data2.add_prefix(column_name)  
                            transformed_data2 = transformed_data2.add_suffix('_current_stay') # Indicate different as temporal over whole of current stay             
                            transformed_data2.insert(0, 'SPELL_IDENTIFIER', stay_id)
                            transformed_data2.insert(1, 'date', new_df2.reset_index().iloc[-1]['date'])

                        else:
                            transformed_data = pd.DataFrame()
                            transformed_data2 = pd.DataFrame()
                        
                        if flag_1 == True:
                            new_df3_2 = new_df3.tail(4).reset_index(drop=True).dropna()
                            new_df3_3 = new_df3.reset_index(drop=True).dropna()

                            # C22 for latest day
                            if len(new_df3_2) <= 2: # c22 only works with 3 or more timepoints
                                transformed_data3 = pd.DataFrame()
                                np_data = new_df3_2.to_numpy()
                                if not np_data.size == 0:
                                    transformed_data3['22'] = np.mean(np_data)
                                    transformed_data3['23'] = np.std(np_data)
                            else: 
                                transformed_data3 = c22.fit_transform(new_df3_2)

                            transformed_data3 = transformed_data3.add_prefix(column_name)           
                            transformed_data3.insert(0, 'SPELL_IDENTIFIER', stay_id)
                            transformed_data3.insert(1, 'date', new_df.reset_index().iloc[-1]['date'])

                            # C22 for all data to date for stay
                            if len(new_df3_3) <= 2: # c22 only works with 3 or more timepoints
                                transformed_data4 = pd.DataFrame()
                                np_data = new_df3_3.to_numpy()
                                if not np_data.size == 0:
                                    transformed_data4['22'] = np.mean(np_data)
                                    transformed_data4['23'] = np.std(np_data) 
                            else: 
                                transformed_data4 = c22.fit_transform(new_df3_3)
                                transformed_data4 = transformed_data4.add_prefix(column_name)  
                                transformed_data4 = transformed_data4.add_suffix('_current_stay') # Indicate different as temporal over whole of current stay             
                                transformed_data4.insert(0, 'SPELL_IDENTIFIER', stay_id)
                                transformed_data4.insert(1, 'date', new_df3.reset_index().iloc[-1]['date'])
                        else:
                            transformed_data3 = pd.DataFrame()
                            transformed_data4 = pd.DataFrame()
                        
                        # Combine
                        transformed_data = pd.concat([transformed_data, transformed_data3], ignore_index=True)
                        transformed_data2 = pd.concat([transformed_data2, transformed_data4], ignore_index=True)

                        # Create master df's
                        master_df = pd.concat([master_df, transformed_data])
                        master_df2 = pd.concat([master_df2, transformed_data2])


                    # This is for most cases  
                    else:
                        # Get index of nth 2 hours grouped value
                        try:
                            index_value = new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index][new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index]['hours_grouped'] == 2].index.values[0]
                            new_df2 = new_df.iloc[:index_value+1]
                        except:
                            # If fails look at nth 1 hours grouped  
                            try:
                                index_value = new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index][new_df.reset_index().iloc[(new_df.reset_index().groupby('hours_grouped').cumcount() == n-1)[new_df.reset_index().groupby('hours_grouped').cumcount() == n-1].index]['hours_grouped'] == 1].index.values[0]
                                new_df2 = new_df.iloc[:index_value+1]
                            except:
                                continue

                        new_df2_2 = new_df2.tail(4).reset_index(drop=True).dropna()
                        new_df2_3 = new_df2.reset_index(drop=True).dropna()
                    
                        # C22 for latest day
                        if len(new_df2_2) <= 2: # c22 only works with 3 or more timepoints
                            transformed_data = pd.DataFrame()
                            np_data = new_df2_2.to_numpy()
                            if not np_data.size == 0:
                                transformed_data['22'] = np.mean(np_data)
                                transformed_data['23'] = np.std(np_data) 
                        else: 
                            transformed_data = c22.fit_transform(new_df2_2)                        
                        
                        transformed_data = transformed_data.add_prefix(column_name)           
                        transformed_data.insert(0, 'SPELL_IDENTIFIER', stay_id)
                        transformed_data.insert(1, 'date', new_df2.reset_index().iloc[-1]['date'])

                        # C22 for all data to date for stay
                        if len(new_df2_3) <= 2: # c22 only works with 3 or more timepoints
                            transformed_data2 = pd.DataFrame()
                            np_data = new_df2_3.to_numpy()
                            if not np_data.size == 0:
                                transformed_data2['22'] = np.mean(np_data)
                                transformed_data2['23'] = np.std(np_data)
                        else: 
                            transformed_data2 = c22.fit_transform(new_df2_3)                        

                        transformed_data2 = transformed_data2.add_prefix(column_name)  
                        transformed_data2 = transformed_data2.add_suffix('_current_stay') # Indicate different as temporal over whole of current stay             
                        transformed_data2.insert(0, 'SPELL_IDENTIFIER', stay_id)
                        transformed_data2.insert(1, 'date', new_df2.reset_index().iloc[-1]['date'])

                        # Create master df's
                        master_df = pd.concat([master_df, transformed_data])
                        master_df2 = pd.concat([master_df2, transformed_data2])

            master_df.reset_index(inplace=True, drop=True)
            master_df2.reset_index(inplace=True, drop=True)

            if master_df.empty:
                if master_df2.empty:
                    continue
                else:
                    master_df = master_df2
            else:
                master_df = master_df.merge(master_df2, how='left', on=['SPELL_IDENTIFIER', 'date'])

        if x == 0:
            overlord_df = master_df.copy()
        elif master_df.empty:
            continue
        else:
            overlord_df = overlord_df.merge(master_df, how='outer', on=['SPELL_IDENTIFIER', 'date'])
            overlord_df.dropna(axis=0, how='all', inplace=True)
        
        # Save as go through
        overlord_df.to_csv('switch_data/new_c22_other_days.csv', index=False)

    return overlord_df

In [12]:
c22_other_days_df = c22_other_fun(c22_other)
# Save
c22_other_days_df.to_csv('switch_data/c22_other_days_final.csv', index=False)

column number: 0
column number: 1
column number: 2
column number: 3
column number: 4
column number: 5
column number: 6
column number: 7
column number: 8
column number: 9


In [19]:
c22_other_days_df.drop(columns=['22_x', '23_x', '22_y', '23_y'], inplace=True)

In [21]:
# Save
#c22_other_days_df.to_csv('switch_data/c22_other_days_final.csv', index=False)

# Combine and work out difference 

In [23]:
# Import
path = r'switch_data/c22_24_hour.csv'
c22_24_hour = pd.read_csv(path)

In [24]:
# Import
path = r'switch_data/c22_48_hour.csv'
c22_48_hour = pd.read_csv(path)

In [27]:
c22_combined = pd.concat([c22_24_hour, c22_48_hour, c22_other_days_df])

In [30]:
# Move column
col = c22_combined.pop("48_hour_flag")
c22_combined.insert(3, col.name, col)

In [70]:
c22_combined2 = c22_combined.reset_index(drop=True)

In [33]:
# Order
c22_combined = c22_combined.sort_values(by=['SPELL_IDENTIFIER', 'date'])

In [73]:
# Difference
c22_combined3 = c22_combined2.drop(columns=['date', '24_hour_flag', '48_hour_flag'])

c22_combined_diff = pd.DataFrame()
c22_combined3.set_index('SPELL_IDENTIFIER', inplace=True)
for stay_id, new_df in c22_combined3.groupby(level=0):
    diff_df = new_df.diff()
    diff_df = diff_df.add_suffix('_difference')
    c22_combined_diff = pd.concat([c22_combined_diff, diff_df], ignore_index=True)

c22_combined3.reset_index(inplace=True)
c22_combined_diff = pd.concat([c22_combined3, c22_combined_diff], axis=1)
c22_combined_diff = pd.concat([c22_combined2[['date', '24_hour_flag', '48_hour_flag']], c22_combined_diff], axis=1)
col = c22_combined_diff.pop("SPELL_IDENTIFIER")
c22_combined_diff.insert(0, col.name, col)

In [75]:
# Save
#c22_combined_diff.to_csv('switch_data/c22_all_with_diff.csv', index=False)

In [3]:
# Import
path = r'switch_data/c22_all_with_diff.csv'
c22_all_with_diff = pd.read_csv(path)

In [5]:
c22_all_with_diff['SPELL_IDENTIFIER'].nunique()

10007